In [ ]:
# !pip install --user --upgrade pip

# !pip install pandas
!pip install spectrum
!pip install pyhrv
# !pip install cvxopt

In [2]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as ss
import sys
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import tools.data_reader_ascertain as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing

from scipy.fft import fft, fftfreq, fftshift

# import warnings
# warnings.filterwarnings(
#     "ignore", 
#     category=RuntimeWarning
# )

In [89]:
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

from biosppy.signals import ecg
import heartpy as hp

convert_sr = False
is_clean_ecg = True
metrics = "bpm"
fs = 256.0
window_size = int(50*fs)

for clip in dr.CLIPS:
    data = []
    for s in dr.SUBJECTS:
        ecg_df = dr.read_ecg(s, clip, "left")
        ecg_signal = ecg_df["ecg"]
        ecg_signal = ecg_signal[-window_size:]
        bpm = preprocessing.get_bpm_biosppy(ecg_signal, fs)

        bpm.insert(0, int(s))
        data.append(bpm)
        
    file_name = os.path.join(dr.Paths.METRICS, f"bpm_Clip{clip}.csv")
    df = pd.DataFrame(data=data)
    df.to_csv(file_name)
        



In [78]:
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

from biosppy.signals import ecg

import pyhrv.time_domain as td

convert_sr = False
is_clean_ecg = True
fs = 256.0
window_size = int(50*fs)

for clip in dr.CLIPS:
# for clip in dr.CLIPS[0:2]:
    metrics_dict = {
        "rmssd": [],
        # "ibi": [],
        "sdnn": []
    }
    for s in dr.SUBJECTS:
    # for s in dr.SUBJECTS[0:5]:
        ecg_df = dr.read_ecg(s, clip, "left")
        ecg_signal = ecg_df["ecg"]
        ecg_signal = ecg_signal[-window_size:]
        # ecg_signal = preprocessing.clean_ecg(ecg_signal, fs).to_numpy().flatten()

        ecg_features = preprocessing.get_ecg_metrics_pyhrv(ecg_signal, fs)

        for metric in list(metrics_dict.keys()):
            feature = ecg_features[metric]
            feature.insert(0, int(s))
            metrics_dict[metric].append(feature)

    for metric in list(metrics_dict.keys()):
        df = pd.DataFrame(metrics_dict[metric])
        metrics_dict[metric] = df
        
    for metric in metrics_dict.keys():
        file_name = os.path.join(dr.Paths.METRICS, f"{metric}_Clip{clip}.csv")
        metrics_dict[metric].to_csv(file_name)
        



In [90]:
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import biosppy
from biosppy.signals import ecg

import heartpy as hp

convert_sr = False
is_clean_ecg = True
fs = 256.0
window_size = int(50*fs)

for clip in dr.CLIPS:
    metrics_dict = {
        "lf_rr": [],
        "hf_rr": [],
    }
    for s in dr.SUBJECTS:
        ecg_df = dr.read_ecg(s, clip, "left")
        ecg_signal = ecg_df["ecg"]
        # ecg_signal = preprocessing.clean_ecg(ecg_signal, fs).to_numpy().flatten()
        out = ecg.ecg(signal=ecg_signal, sampling_rate=fs, show=False)
        ecg_signal = out["filtered"]
        ecg_signal = ecg_signal[-window_size:]
        
        lf_rr = preprocessing.get_lf_rr(ecg_signal, fs)
        lf_rr = np.insert(lf_rr, 0, s)
        lf_rr = pd.DataFrame(lf_rr).dropna(axis=1)

        hf_rr = preprocessing.get_hf_rr(ecg_signal, fs)
        hf_rr = np.insert(hf_rr, 0, s)
        hf_rr = pd.DataFrame(hf_rr).dropna(axis=1)

        metrics_dict["lf_rr"].append(lf_rr)
        metrics_dict["hf_rr"].append(hf_rr)
        
    for metric in list(metrics_dict.keys()):
        df = pd.concat(metrics_dict[metric], axis=1)
        df = df.transpose()
        metrics_dict[metric] = df

    for metric in list(metrics_dict.keys()):
        file_name = os.path.join(dr.Paths.METRICS, f"{metric}_Clip{clip}.csv")
        metrics_dict[metric].to_csv(file_name)

In [80]:
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import biosppy
from biosppy.signals import eda

convert_sr = False
is_clean_ecg = True
fs = 100.0
window_size = int(50*fs)

for clip in dr.CLIPS:
    metrics_dict = {
        "mean_SCL": [],
        "SCR_rate": [],
    }
    for s in dr.SUBJECTS:
        eda_df = dr.read_gsr(s, clip, "x")
        eda_signal = eda_df["eda"]
        # ecg_signal = preprocessing.clean_ecg(ecg_signal, fs).to_numpy().flatten()
        out = eda.eda(signal=eda_signal, sampling_rate=fs, show=False)
        eda_signal = out["filtered"]
        eda_signal = eda_signal[-window_size:]
        
        mean_scl = preprocessing.get_mean_SCL(eda_signal, fs)
        mean_scl = np.insert(mean_scl, 0, s)
        mean_scl = pd.DataFrame(mean_scl).dropna(axis=1)

        scr_rate = preprocessing.get_SCR_rate(eda_signal, fs)
        scr_rate = np.insert(scr_rate, 0, s)
        scr_rate = pd.DataFrame(scr_rate).dropna(axis=1)

        metrics_dict["mean_SCL"].append(mean_scl)
        metrics_dict["SCR_rate"].append(scr_rate)
        
    for metric in list(metrics_dict.keys()):
        df = pd.concat(metrics_dict[metric], axis=1)
        df = df.transpose()
        metrics_dict[metric] = df

    for metric in list(metrics_dict.keys()):
        file_name = os.path.join(dr.Paths.METRICS, f"{metric}_Clip{clip}.csv")
        metrics_dict[metric].to_csv(file_name)

mean SCL: -0.006594962815638722
SCR rate: 4
mean SCL: -0.04247432614574194
SCR rate: 4
mean SCL: -0.042785962596969304
SCR rate: 3
mean SCL: 0.02238657576153962
SCR rate: 4
mean SCL: -0.04370262125842516
SCR rate: 3
mean SCL: -0.04379191253593769
SCR rate: 4
mean SCL: 0.03766829152116248
SCR rate: 4
mean SCL: -0.001910756558441097
SCR rate: 2
mean SCL: -0.03802727423003056
SCR rate: 5
mean SCL: -0.0103695633181954
SCR rate: 2
mean SCL: 0.024923340040318216
SCR rate: 6
mean SCL: 0.03383149987978399
SCR rate: 3
mean SCL: -0.02211170117047218
SCR rate: 3
mean SCL: 0.02979617649501791
SCR rate: 3
mean SCL: -0.003239910300229509
SCR rate: 4
mean SCL: 0.03861265612519222
SCR rate: 3
mean SCL: 0.035096171497058445
SCR rate: 4
mean SCL: -0.04315031523716836
SCR rate: 5
mean SCL: -0.03734651013716904
SCR rate: 3
mean SCL: -0.03762532703053942
SCR rate: 3
mean SCL: 0.02726379505947013
SCR rate: 2
mean SCL: -0.03048856757693638
SCR rate: 3
mean SCL: -0.0012241617506785676
SCR rate: 3
mean SCL: 0.

In [ ]:
# STATISTICAL FEATURES - entropy, variance, kurtosis, skewness
from biosppy.signals import eda
from scipy import stats, square


fs = 100.0
window_size = int(50*fs)
metric_fn = stats.kurtosis

for clip in dr.CLIPS:
    print(clip + " " + "-"*80)
    metrics = []
    for s in dr.SUBJECTS:
        metric = [int(s)]
        eda_df = dr.read_gsr(s, clip, "x")
        eda_signal = eda_df["eda"]
        out = eda.eda(signal=eda_signal, sampling_rate=fs, show=False)
        eda_signal = out["filtered"]

        segment = eda_signal[-window_size:]
        metric.append(metric_fn(segment))
        metrics.append(metric)
    df = pd.DataFrame(metrics)
    file_name = os.path.join(dr.Paths.METRICS, f"kurtosis_Clip{clip}.csv")
    df.to_csv(file_name)

In [65]:
# FEATURES - Ten low frequency ([0-2.4] Hz) power spectral densities (PSDs), four very slow
# response ([0-0.04] Hz) PSDs, IBI, HR, and HRV statistics

import scipy.io as sio

from biosppy.signals import eda
from scipy import stats, square


file = os.path.join(dr.Paths.DATA_DIR, "ASCERTAIN", f"Dt_ECGFeatures.mat")
mat = sio.loadmat(file)["ECGFeatures_58"][0, :]
n_features = mat[0].shape[1]

features = []
for i, clip in enumerate(dr.CLIPS):
    # print(clip + " " + "-"*80)
    metrics = []
    for j, s in enumerate(dr.SUBJECTS):
        data = list(mat[j][i])
        data.insert(0, int(s))
        features.append(data)

features = pd.DataFrame(features, columns=["subject"] + [i for i in range(1, n_features+1)])
features = features.drop(list(range(1, 18)), axis=1)
print(features.iloc[0:20, :])

    subject   18         19         20        21        22         23  \
0         1  0.0   6.400031   6.444472  5.173039  5.276134   5.462701   
1         2  0.0  -3.223879  -0.894268 -2.459274 -1.888241  -4.907992   
2         3  0.0   2.522513   3.087524  2.955319  2.346282   1.991922   
3         4  0.0   9.055861   7.717121  6.583570  5.392990   9.106767   
4         5  0.0   7.602714   7.907524  5.680512  5.677677   6.610141   
5         6  0.0   3.436547   3.984934  3.375845  3.906665   3.394669   
6         7  0.0  10.564949  10.668652  9.016305  8.699533  10.349783   
7         8  0.0   8.006420   8.442141  5.612111  5.216608   6.234937   
8         9  0.0   4.353876   5.017725  3.273353  7.081505   3.070474   
9        10  0.0   4.571603   4.737512  5.341372  5.879296   3.536648   
10       11  0.0   5.862254   7.038705  7.225714  7.584943   5.192760   
11       12  0.0  10.282349  10.373724  8.808814  8.403514   9.178007   
12       13  0.0  10.221121  10.513929  7.891484  7